# Parkinson Dataset
## Γενικές πληροφορίες για το σετ δεδομένων

Αυτό το σετ δεδομένων αποτελείται από ηχητικές ιατρικές μετρήσεις 42 ανθρώπων με την ασθένεια Parkinson σε αρχικό στάδιο. Οι άνθρωποι αυτοί προσλήφθηκαν για μια εξάμηνη δοκιμή μιας συσκευής τηλεπαρακολούθησης για εξ αποστάσεως διάγνωση της ασθένειας. Τα δεδομένα καταγράφονταν αυτόματα στα σπίτια των ασθενών.

Οι στήλες περιλαμβάνουν έναν index number του κάθε ασθενή, την ηλικία του, το φύλο του, την χρονική διάρκεια από την αρχική ημερομηνία πρόσληψης, δύο δείκτες motor UPDRS και total UPDRS, και 16 άλλες ιατρικές ηχητικές μετρήσεις. Κάθε γραμμή σχετίζεται με ένα εκ των 5,875 διαφορετικών ηχητικών μετρήσεων από αυτούς του εθελοντές. **Ο κύριος στόχος μας είναι να προβλέψουμε τους δείκτες UPDRS (δηλαδή το 'motor_UPDRS' και 'total_UPDRS') από τις 16 ηχητικές μετρήσεις.**

Μερικά ακόμα στοιχεία που μας δίνονται για το σετ δεδομένων είναι τα παρακάτω:

* subject - Integer that uniquely identifies each subject
* age - Subject age
* sex - Subject gender '0' - male, '1' - female
* test_time - Time since recruitment into the trial. The integer part is the 
* number of days since recruitment.
* motor_UPDRS - Clinician's motor UPDRS score, linearly interpolated
* total_UPDRS - Clinician's total UPDRS score, linearly interpolated
* Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP - Several measures of 
* variation in fundamental frequency
* Shimmer,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA - 
* Several measures of variation in amplitude
* NHR,HNR - Two measures of ratio of noise to tonal components in the voice
* RPDE - A nonlinear dynamical complexity measure
* DFA - Signal fractal scaling exponent
* PPE - A nonlinear measure of fundamental frequency variation

Οπότε αρχικά φορτώνουμε κάποιες χρήσιμες βιβλιοθήκες και το σετ δεδομένων μας.

In [ ]:
import pandas as pd
import scipy as sc
import numpy as np
from sklearn.preprocessing import MinMaxScaler

Βλέπουμε ότι στο σύνολο δεδομένων μας υπάρχει ένα feature με το όνομα `subject#`, το οποίο ουσιαστικά μας δείχνει ποιός ασθενής είναι ο καθένας. Η διάταξη του όμως δεν έχει κάποιο νόημα (είναι μη διατεταγμένο), οπότε μπορεί να μπερδέψει στην συνέχεια την εκπαίδευση των ταξινομητών μας. Θα πρέπει λοιπόν να βρούμε έναν τρόπο να το χειριστούμε έτσι ώστε να μας δίνει μια πληροφορία που να έχει κάποιο νοημα.

Μπορούμε είτε να μετατρέψουμε το feature αυτό με one-hot encoding είτε να το πετάξουμε τελείως. Η πρώτη μέθοδος θα μας πρόσθετε υπερβολικά πολλά χαρακτηριστικά, οπότε θα δοκιμάσουμε την δεύτερη. 

In [ ]:
dataset = pd.read_csv('../input/parkinsons-telemonitoring-data/telemonitoring_parkinsons_updrs.data.csv')
dataset[0:30]

Στην συνέχεια βλέπουμε ότι υπάρχουν ονοματά στις στήλες (δηλαδή στα διαφορετικά features). Βλέπουμε συγκεκριμένα ότι έχουμε 22 διαφορετικές features και 5875 το πλήθος εγγραφές.

In [ ]:
print('Data Column Names:',dataset.columns)
print('Length of Column Name List:',len(dataset.columns))

In [ ]:
print(dataset.shape)

Πετάμε τελείως την μεταβλητή `subject#` γιατί είναι μη διατεταγμένη και μπορεί να μπερδέψει τα δεδομένα μας.

In [ ]:
dataset = dataset.drop(['subject#'], axis=1)

Πριν κάνουμε οτιδήποτε, πρέπει αν σιγουρευτούμε ότι δεν υπάρχουν missing values σε διάφορα χαρακτηριστικά. Αυτό είναι ένα σημαντικό βήμα, γιατί σε περίπτωση που υπάρχουν πρέπει να αποφασήσουμε πως θα πρέπει να τα χειριστούμε. Μπορούμε ας πούμε να τα διαγράψουμε είτε αν ακολουθήσουμε άλλες τεχνικές όπως είναι αυτές του Expectation Maximization (EM) ή το pseudo-EM.

Στην δικιά μας περίπτωση ευτυχώς δεν υπάρχουν καθόλου missing values, οπότε δεν θα μας απασχολήσει αυτό το πρόβλημα.

In [ ]:
dataset.isnull().sum()

Χωρίζουμε τον πίνακα σε στοιχεία εισόδου και εξόδου 

X: Πίνακας Χαρακτηριστικών 

Y: Πίνακας Ετικετών

In [ ]:
array = dataset.values
X1 = array[:,0:4]
X2 = array[:,6:]
X = np.hstack((X1,X2))
Y = array[:,4:6]

In [ ]:
X.shape

In [ ]:
Y.shape

Στην συνέχεια χωρίζουμε τα δεδομένα μας σε train και test, όπως φαίνεται παρακάτω. Φροντίζουμε το train set μας να είναι το 30% του αρχικού σετ δεδομένων μας.

In [ ]:
# load and summarize the dataset
from sklearn.model_selection import train_test_split

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

## Feature Selection
### Correlation
Η συσχέτιση (correlation) είναι ένα μέτρο που μας δείχνει εάν δύο τυχαίες μεταβλητές εξαρτώνται η μια από την άλλη. Το πιο κοινότυπο μέτρο συσχέτισης είναι αυτό της συσχέτισης Pearson, το οποίο υποθέτει ότι οι τυχαίες μας μεταβλητές κατανέμονται με Γκαουσιανή κανονική κατανομή και αναφέρεται στην γραμμική τους εξάρτηση.

Τα μέτρα γραμμικής συσχέτισης είναι συνήθως στο διάστημα από -1 εώς 1 με το 0 να σημαίνει μη συσχέτιση. Για την εξαγωγή χαρακτησριστικών, ενδιαφερόμαστε να έχουμε ένα θετικό δείκτη. Όσο πιο μεγάλος είναι ο δέικτης συγκριτικά με την τυχαία μεταβλητή που θέλουμε να προβλέψουμε, σημαίνει ότι τα χαρακτηριστικά μας, είναι πολύ συσχετισμένες με αυτήν. Άρα έχουμε περισσότερες ελπίδες να φτιάξουμε ένα καλό μοντέλο με τέτοιου τύπου τυχαίες μεταβλητές.

Ένας τρόπος να δούμε πόσο συσχετισμένες είναι οι τυχαίες μεταβλητές μας είναι με f-score. Το f-score είναι ένα μέτρο συσχέτισης, και προφανώς όσο πιο μεγάλο είναι για ένα χαρακτηριστικό, τόσοπιο πιθανό είναι να το κρατήσουμε στο τελικό μας μοντέλο.

Στην δικιά μας περίπτωση θέλουμε να προβλέψουμε δύο τυχαίες μεταβλητές y, αυτό σημαίνει ότι θα πρέπει να δούμε τις συσχετίσεις με κάθε μια από αυτές. Αρχικά λοιπόν μπορούμε να κάνουμε και ένα bar-plot με τα f-scores την πρώτη μεταβλητή motor UPDRS.

In [ ]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from matplotlib import pyplot
 
# feature selection
def select_features(X_train, y_train, X_test):
	# configure to select all features
	fs = SelectKBest(score_func=f_regression, k='all')
	# learn relationship from training data
	fs.fit(X_train, y_train)
	# transform train input data
	X_train_fs = fs.transform(X_train)
	# transform test input data
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs

# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train[:,0], X_test)
# what are scores for the features
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
pyplot.bar([i for i in range(len(fs.scores_))], fs.scores_)
pyplot.show()

Στην συνέχεια κάνουμε και ένα δεύτερο bar plot για την τυχαία μεταβλητή total UPDRS.

In [ ]:
# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train[:,1], X_test)
# what are scores for the features
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
pyplot.bar([i for i in range(len(fs.scores_))], fs.scores_)
pyplot.show()

### Mutual Information Feature Selection

Πρόκειται για ένα μέτρο που προέρχεται από την θεωρία πληροφορίας. Το mutual information υπολογίζεται μεταξύ δύο μεταβλητών και μετράει την μείωση της αβεβαιότητας για μια τυχαία μεταβλητή, θεωρόντας γνωστή την τιμή μιας άλλης τυχαίας μεταβλητής.

Μπορεί να χρησιμοποιηθεί με παρόμοιο τρόπο όπως και το f-score. Επομένως μπορούμε να θεωρήσουμε ότι τα k καλύτερα χαρακτηριστικά είναι αυτά με το μεγαλύτερο mutual information.

In [ ]:
# example of mutual information feature selection for numerical input data
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from matplotlib import pyplot

# feature selection
def select_features(X_train, y_train, X_test):
	# configure to select all features
	fs = SelectKBest(score_func=mutual_info_regression, k='all')
	# learn relationship from training data
	fs.fit(X_train, y_train)
	# transform train input data
	X_train_fs = fs.transform(X_train)
	# transform test input data
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs
 
# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train[:,0], X_test)
# what are scores for the features
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
pyplot.bar([i for i in range(len(fs.scores_))], fs.scores_)
pyplot.show()

In [ ]:
# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train[:,1], X_test)
# what are scores for the features
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
pyplot.bar([i for i in range(len(fs.scores_))], fs.scores_)
pyplot.show()

### Filter method (Ηeatmap):
Από την μια πλευρά ζητάμε να έχουμε συσχέτιση μεταξύ των features και των εξαρτημένων μεταβλητών y. Όμως από την άλλη μεριά (τουλάχιστον για να ισχύουν οι υποθέσεις του γραμμικού μοντέλου) θέλουμε τα feautures να είναι μεταξύ τους όσο το δυνατόν περισσότερο ασυσχέτιστα.

Ασυσχέτιστα είναι τα features εάν έχουν τιμές κοντά στο μηδέν. Αν έχουν υψηλά correlations κατά απόλυτη τιμή (είτε θετικά είτε αρνητικά) τότε υπάρχει γενικά πρόβλημα. Πρέπει να προσπαθήσουμε να κρατήσουμε features με όσο το δυνατόν χαμηλότερες μεταξύ τους συσχετίσεις.

Ένας τρόπος να δούμε τις συσχετίσεις μεταξύ των δεδομένων μας είναι με το heatmap.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = dataset.corr()
sns.heatmap(cor, annot=False, cmap=plt.cm.Reds)
plt.show()

Επομένως μπορούμε να βάλουμε ένα threshold και να δούμε ποιές από τις τυχαίες μεταβλητές μας είναι περισσότερο θετικά συσχετισμένες με τις τυχαίες μεταβλητές y που θέλουμε να προβλέψουμε.

Στην προκειμένη περίπτωση βάζουμε ως threshlold το 0.1. Αυτός είναι και ένας τρόπος να κάνουμε επιλογή χαρακτηριστικών. Βέβαια με αυτόν τον τρόπο βλέπουμε πόσο συσχετισμένα είναι τα features με την y, όμως δεν ξέρουμε πόσο συσχετισμένα είναι μεταξύ τους. 

In [ ]:
#Correlation with output variable
cor_target = abs(cor['motor_UPDRS'])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.1]
relevant_features

In [ ]:
#Correlation with output variable
cor_target = abs(cor['total_UPDRS'])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.1]
relevant_features

### Wrapper Method:
Μια άλλη μέθοδος είναι να χρησιμοποιήσουμε έναν ταξινομητή machine learning και να τον δοκιμάσουμε επαναληπτικά σε υποσύνολα του αρχικού μας συνόλου δεδομένου, μέχρι να βρούμε σε ποιό από αυτά κάνει την καλύτερη πρόβλεψη. Πρόκειται για μια πιο ακριβή υπολογιστικά μέθοδο, αλλά είναι πιο ακριβής από τις προηγούμενες. Στην συγκεκριμένη περίπτωση χρησιμοποιούμε τον ταξινομητή oridnary least squares.

Υπάρχουν αρκετοί τρόποι να κάνουμε αυτήν την διαδικασία. Εδώ θα χρησιμοποιήσουμε δύο από αυτές: το backward elimintaion και το RFE. 

#### i. Backward Elimination
Όπως σημαίνει και το όνομα, αυτό που κάνουμε είναι να ξεκινάμε με ένα μοντέλο που περιέχει όλα όλα τα features και στην συνέχεια αρχίζουμε να αφαιρούμε επαναληπτικά ένα ένα αυτά τα features εξετάζοντας τον αλγόριθμο μηχανικής μάθησης που έχουμε επιλέξει, πότε έχει την καλύτερη δυνατή απόδοση.

Εδώ εξετάζουμε την απόδοση του αλγορίθμου μας μέσω της μετρικής του p-value. Αν το p-value είναι μεγαλύτερο από 0.05 τότε αφαιρούμε το χακακτηριστικό, αλλιώς το κρατάμε.

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

#Adding constant column of ones, mandatory for sm.OLS model
X_1 = sm.add_constant(X_train)
#Fitting sm.OLS model
model = sm.OLS(y_train[:,0],X_1).fit()
model.pvalues

Παραπάνω έχουμε κάνει ένα iteration. Θα αφαιρέσουμε χαρακτηριστικά με p-value>0.05. Μπορούμε να δημιουργήσουμε ένα loop και να κάνουμε επαναληπτικά αυτήν την διαδικασία μέχρι να καταλήξουμε στο καλύτερο δυνατό μοντέλο.

In [ ]:
#Backward Elimination
cols = list(pd.DataFrame(X_train).columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = pd.DataFrame(X_train)[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(pd.DataFrame(y_train[:,0]),X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

#### ii. RFE (Recursive Feature Elimination)

Εδώ πάλι αφαιρούμε χαρακτηριστικά μέχρις ότου να φτάσουμε στο βέλτιστο δυνατό μοντέλο. Η κύρια διαφορά είναι ότι πλέον χρησιμποιύμε accuracy score. Η RFE μέθοδος παίρνει ως input τον αριθμό των features που θέλουμε να έχουμε στο τελικό μας μοντέλο, καθώς και έναν ταξινομητή μέσω του οποίου θα υπολογίσουμε το accuracy score. Ως έξοδο μας εμφανίζει ένα ranking μεταξύ όλων των feautures που έχει κρατήσει.

In [ ]:
model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 7)
#Transforming data using RFE
X_rfe = rfe.fit_transform(X_train,y_train[:,0])  
#Fitting the data to model
model.fit(X_rfe,y_train[:,0])
print(rfe.support_)
print(rfe.ranking_)

### Embedded Method
Οι embedded methods είναι μέθοδοι για να κρατάμε προσεκτικά χαρατηριστικά που συμβάλουν περισσότερο σε κάθε iteration. Ένας τρόπος για να το κάνουμε αυτό είναι χρησιμοποιώντας regularization χρησιμοποιώντας ένα threshold ενός συντελεστή-δείκτη. Συγκεκριμένα θα χρησιμοποιήσουμε Lasso regularization. Αν ένα feature δεν σχετίζεται με το μοντέλο μας, βάζουμε συντελεστή 0 στο lasso. Και με αυτόν τον τρόπο όλα τα χαρακτηριστικά με συντελεστή 0 τα πετάμε από το μοντέλο μας.

In [ ]:
reg = LassoCV()
reg.fit(X_train, y_train[:,0])
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X_train,y_train[:,0]))
coef = pd.Series(reg.coef_, index = pd.DataFrame(X_train).columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Model")

## Επιλογή χαρακτηριστικών
Μπορούμε τώρα έχοντας κάνει όλη την παραπάνω δουλεία να διαλέξουμε ένα υποσύνολο των πιο σημαντικών features με τα οποία θα δουλέψουμε. Αυτό θα το κάνουμε κρατώντας μόνο εκείνα τα χαρακτηριστικά, τα οποία βρήκαμε στο προηγούμενο backward elimination.

In [ ]:
X_train = pd.DataFrame(X_train)
X_train = X_train[[0, 1, 2, 3, 4, 6, 9, 11, 12, 15, 16, 17, 18]]
X_train = X_train.values

X_test = pd.DataFrame(X_test)
X_test = X_test[[0, 1, 2, 3, 4, 6, 9, 11, 12, 15, 16, 17, 18]]
X_test = X_test.values

Παρόλα αυτά, όπως βλέπουμε και παρακάτω, ακόμα υπάρχουν προβλήματα. Υπάρχουν ακόμα χαρακτηριστικά τα οποία είναι μεταξύ τους correlated. Συγκεκριμένα εάν δούμε τα `Shimmer, Shimmer(dB), Shimmer:APQ3, Shimmer:APQ5, Shimmer:APQ11, Shimmer:DDA`, αυτά έχουν στενές συσχετίσεις μεταξύ τους, κάτι που φαίνεται αρκετά λογικό, αφού μοιάζει να περιγράφουν παρόμοια πράγματα.

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = pd.DataFrame(X_train).corr()
sns.heatmap(cor, annot=False, cmap=plt.cm.Reds)
plt.show()

## Εκπαίδευση μοντέλων
Σε αυτό το μέρος θα εκπαιδεύσουμε κάποιους παλινδρομητές, έτσι ώστε να μάθουν από τα δεδομένα μας και να κάνουν πρόβλεψη. Το πρώτο πράγμα που κάνουμε είναι να φορτώσουμε όλες τις βιβλιοθήκες της `sklearn` που θα μας χρειαστούν και στην συνέχεια.

Ορίζουμε επίσης έναν scaler, και μια κλάση PCA που θα μας χρειαστούν στην συνέχεια για να κάνουμε το grid search με cross validation. Επίσης έχουμε δημιουργήσει μια συνάρτηση `compute_metrics`, η οποία δεδομένου ενός ταξινομητή που τον έχουμε κάνει fit σε κάποια δεδομένα, μας κάνει προβλέψεις, και υπολογίζει τις μετρικές $R^{2}$ και MSE για τα σύνολα εκπαίδευσης και ελέγχου.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
import time

# αρχικοποιούμε τους μετασχηματιστές χωρίς υπερ-παραμέτρους
scaler = StandardScaler()
pca = PCA()

n_components = [3, 5, 7, 8, 10, 12, 13]

def compute_metrics(model, x_tr, y_tr, x_ts, y_ts):
    train_predict = model.predict(x_tr)
    test_predict = model.predict(x_ts)

    # Υπολογισμός Ορθότητας (Accuracy)
    train_accuracy = r2_score(y_tr, train_predict)
    test_accuracy = r2_score(y_ts, test_predict)

    # Υπολογισμός Μέσου Τετραγωνικού Σφάλματος
    train_MSE = mean_squared_error(y_tr, train_predict)
    test_MSE = mean_squared_error(y_ts, test_predict)

    print('Ορθότητα στο σύνολο δεδομένων εκπαίδευσης: {:.2%}'.format(
      train_accuracy))
    print('Ορθότητα στο σύνολο δεδομένων ελέγχου: {:.2%}\n'.format(test_accuracy))
    print('Μέσο Τετραγωνικό Σφάλμα στα δείγματα εκπαίδευσης: {:.4f}'.format(
      train_MSE))
    print('Μέσο Τετραγωνικό Σφάλμα στα δείγματα ελέγχου: {:.4f}'.format(test_MSE))
    return train_predict, test_predict, train_accuracy, test_accuracy, train_MSE, test_MSE

Εδώ ορίζουμε κάποια διανύσματα τα οποία βασικά θα μας χρειαστούν στην συνέχεια για να σχεδιάσουμε τα bar plots, που θα αποθηκεύουν τους χρόνους κάθε αλγορίθμου, τους χρόνους που χρειαζόμαστε για να τους κάνουμε tuning και τις μετρικές που βρίσκουμε από αυτούς.

In [ ]:
names = ['Linear','Polynomial', 'Elastic-Net', 'Lasso', 'Tree', 'kNN', 'Forest', 'BGR']
times = []
CV_times = []
r2 = []
MSE = []

### Μέρος 1ο: Γραμμικά μοντέλα
Αρχικά θα χρησιμοποιήσουμε γραμμικά μοντέλα. Είναι τα πιο κλασικά μοντέλα που μαθαίνει κανείς σε ένα πρώτο μάθημα στατιστικών προτύπων για να κάνει παλινδόμιση. Ας υπογραμμίσουμε ότι τόσο για το την γραμμική παλινδρόμιση όσο και για την πολυωνυμική παλινδρόμιση δεν χρειάζεται να κάνουμε tuning κάποια υπερπαράμετρο, μιας και για τον υπολογισμό των συντελεστών αυτών των μοντέλων το μόνο που πραγματικά που χρειάζεται είναι να κάνουμε ελαχιστοποίηση μιας συνάρτησης κόστους.

#### Linear Regression

In [ ]:
start_time = time.time()
clf = MultiOutputRegressor(LinearRegression())
clf.fit(X_train, y_train)
train_predict, test_predict, train_accuracy, test_accuracy, train_MSE, test_MSE = compute_metrics(clf, X_train, y_train, X_test, y_test)
times.append(time.time() - start_time)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))

CV_times.append(0)
r2.append(test_accuracy)
MSE.append(test_MSE)

#### Polynomial Regression

In [ ]:
pf2 = PolynomialFeatures(degree=2)

x_train2 = pf2.fit_transform(X_train)
x_test2 = pf2.fit_transform(X_test)

start_time = time.time()
clf = lr = MultiOutputRegressor(LinearRegression())
lr.fit(x_train2, y_train)

train_predict, test_predict, train_accuracy, test_accuracy, train_MSE, test_MSE = compute_metrics(clf, x_train2, y_train, x_test2, y_test)
times.append(time.time() - start_time)

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))

CV_times.append(0)
r2.append(test_accuracy)
MSE.append(test_MSE)

#### Elastic-Net
Στο elastic-net τώρα υπάρχουν κάποιες υπερπαράμετροι να ρυθμίσουμε. Για αυτόν τον λόγο κάνουμε ένα grid search με cross validation για να βρούμε τις βέλτιστες. Και βλέπουμε πράγματι ότι μετά από αυτήν την διαδικασία η ικανότητα πρόβλεψης και γενίκευσης του αλγορίθμου μας βελτιώνεται δραματικά. 

In [ ]:
start_time = time.time()
en = ElasticNet()
en.fit(X_train,y_train)
compute_metrics(en, X_train, y_train, X_test, y_test)
times.append(time.time() - start_time)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))

In [ ]:
l1 = np.arange(0.1,1,0.2)
alphas = np.arange(0,2,0.1)
pipe_en = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('ElasticNet', en)], memory = 'tmp')
estimator_en = GridSearchCV(pipe_en, dict( pca__n_components=n_components, ElasticNet__alpha=alphas , ElasticNet__l1_ratio=l1), cv=5, scoring ='r2', n_jobs=-1)
start_time = time.time()
estimator_en.fit(X_train, y_train)
train_predict, test_predict, train_accuracy, test_accuracy, train_MSE, test_MSE = compute_metrics(estimator_en, X_train, y_train, X_test, y_test)
CV_times.append(time.time() - start_time)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))

r2.append(test_accuracy)
MSE.append(test_MSE)

#### Lasso regression
Παρόμοια με τα προηγούμενα εργαζόμαστε και για την παλινδρόμιση του Lasso. Βλέπουμε και πάλι δραματική βελτίωση στην απόδοση μετά την διαδικασία του grid search.

In [ ]:
start_time = time.time()
lasso = linear_model.MultiTaskLasso()
lasso.fit(X_train, y_train)
compute_metrics(lasso, X_train, y_train, X_test, y_test)
times.append(time.time() - start_time)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))

In [ ]:
alphas = np.arange(0.1,2,0.1)
pipe_lasso = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('lasso', lasso)], memory = 'tmp')
estimator_lasso = GridSearchCV(pipe_lasso, dict( pca__n_components=n_components, lasso__alpha=alphas ), cv=5, scoring ='r2', n_jobs=-1)
start_time = time.time()
estimator_lasso.fit(X_train, y_train)
train_predict, test_predict, train_accuracy, test_accuracy, train_MSE, test_MSE = compute_metrics(estimator_lasso, X_train, y_train, X_test, y_test)
CV_times.append(time.time() - start_time)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))

r2.append(test_accuracy)
MSE.append(test_MSE)

### Μέρος 2ο: Decission tree and k-Nearest Neighbors

Το Decission tree είναι ένας τρόπος να κάνουμε παλινδρόμιση, θέτοντας ερωτήσεις για τα δεδομένα μας. Τα δέντρα χρησιμοποιούν δύο συναρτήσεις impurity και total impurity έτσι ώστε να κάνουν τις 'βέλτιστες ερωτήσεις' για τα δεδομένα μας. Είναι μη γραμμικοί ταξινομητές, και έχουν διάφορες υπερπαραμέτρους όπως είναι ο συντελεστής CCP-a, το βάθος τους, ο αριθμός των nodes των φύλλων τους. Επίσης μπορούμε σε περίπτωση που έχει γίνει overfitting, να κάνουμε κλάδεμα του δέντρου έτσι ώστε να μην μαθαίνει υπερβολικά καλά τα δεδομένα εκπαίδευσης. 

Ο παλινδρομιτής kNN κάνει παλινδρόμιση, βλέποντας τις τιμές που έχουν οι κοντινότεροι γείτονες ενός στοιχείου στον χώρο των δεδομένων μας. Οι κύριες υπερπαράμετροι έχουν να κάνουν με τον αλγόριθμο τον οποίο μετράμε αποστάσεις στον χώρο των δεδομένων μας, καθώς και τον αριθμό των γειτόνων που κοιτάμε κάθε φορά.

#### Decission Tree Regressor
Αυτός ο παλινδρομιτής παίρνει αρκετή ώρα για να tunarίστεί, επομένως κάναμε tuning μόνο στην παράμετρο CCP-a. Και πράγματι βελτιώνεται πολύ λίγο, βλέπουμε βέβαια ότι και χωρίς να κάνουμε grid search πετυχαίνουμε πολύ υψηλά scores. Αυτό προφανώς συμβαίνει γιατί τα δέντρα είναι πολύ ισχυροί μη γραμμικοί ταξινομητές, οπότε δεν χρειάζεται να κάνουμε πολύ tuning. 

Ας υπογραμμίσουμε επίσης ότι δεν κάναμε PCA. Το δοκιμάσαμε αλλά είδαμε ότι με PCA το grid search  βγάζει χειρότερα αποτελέσματα.

In [ ]:
start_time = time.time()
tree = MultiOutputRegressor(DecisionTreeRegressor(random_state=0))
tree.fit(X_train, y_train)
compute_metrics(tree, X_train, y_train, X_test, y_test)
times.append(time.time() - start_time)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))

In [ ]:
alphas = np.arange(0.0,2.0,0.2)
pipe_tree = Pipeline(steps=[('scaler', scaler), ('tree', tree)], memory = 'tmp')
treeCV = GridSearchCV(pipe_tree, dict(tree__estimator__ccp_alpha=alphas ), cv=5, scoring ='r2', n_jobs=-1)

start_time = time.time()
treeCV.fit(X_train, y_train)

train_predict, test_predict, train_accuracy, test_accuracy, train_MSE, test_MSE = compute_metrics(treeCV, X_train, y_train, X_test, y_test)
CV_times.append(time.time() - start_time)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))

r2.append(test_accuracy)
MSE.append(test_MSE)

#### k-Nearest neighbors
Όσον αφορά τον kNN κάνουμε tuning δύο υπερπαραμέτρους που έχουν να κάνουν με τον αλγόριθμο που μετράει αποστάσεις, καθώς και τον αριθμό των κοντινότερων γειτόνων που χρησιμοποιούμε για να μάθουμε τα δεδομένα μας. Πάλι το PCA περισσότερο εμποδίζει, για αυτό από εδώ και στο εξής το έχουμε βγάλει. Βλέπουμε πάντως ότι η απόδοση του αλγορίθμου βελτιώνεται πάρα πολύ.

In [ ]:
neigh = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3))
start_time = time.time()
neigh.fit(X_train, y_train)
compute_metrics(neigh, X_train, y_train, X_test, y_test)
times.append(time.time() - start_time)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))

In [ ]:
algos = ['auto', 'ball_tree', 'kd_tree', 'brute']
num = [1,2,3,4,5,6]
pipe_neigh = Pipeline(steps=[('scaler', scaler), ('neigh', neigh)], memory = 'tmp')
neighCV = GridSearchCV(pipe_neigh, dict( neigh__algorithm =algos, neigh__n_neighbors=num ), cv=5, scoring ='r2', n_jobs=-1)

start_time = time.time()
treeCV.fit(X_train, y_train)
train_predict, test_predict, train_accuracy, test_accuracy, train_MSE, test_MSE = compute_metrics(treeCV, X_train, y_train, X_test, y_test)
CV_times.append(time.time() - start_time)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))

r2.append(test_accuracy)
MSE.append(test_MSE)

### Μέρος 3ο: Μοντέλα Boosting-Ensemble
Τα Random Forests είναι αλγόριθμοι που θυμίζουν πάρα πολύ τα δέντρα. Στην πραγματικότητα είναι συνδυασμοί από πολλά δέντρα μαζί και στην ουσία έχουν παρόμοιες υπερπαραμέτρους, για αυτό και εργαζόμαστε παρόμοια με τα δέντρα.

Όσον αφορά το gradient boosting καταφεύγουμε στον ορισμό της Βικιπαίδειας:

> Gradient boosting is a machine learning technique for regression and classification problems, which produces a prediction model in the form of an ensemble of weak prediction models, typically decision trees.[1][2] When a decision tree is the weak learner, the resulting algorithm is called gradient boosted trees, which usually outperforms random forest. It builds the model in a stage-wise fashion like other boosting methods do, and it generalizes them by allowing optimization of an arbitrary differentiable loss function.

####  Random Forest
Όπως και στα δέντρα η απόδοση είναι ήδη πολύ υψηλή χωρίς τιουνάρισμα, οπότε μετα'την πλεγματική αναζήτηση βλεουμε πολύ μικρή αύξηση στην απόδοση.

In [ ]:
rd_for = MultiOutputRegressor(RandomForestRegressor(random_state=0))
start_time = time.time()
rd_for.fit(X_train, y_train)
compute_metrics(rd_for, X_train, y_train, X_test, y_test)
times.append(time.time() - start_time)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))

In [ ]:
alphas = np.arange(0,1,0.2)
pipe_rdfor = Pipeline(steps=[('scaler', scaler), ('rd_for', rd_for)], memory = 'tmp')
rdforCV = GridSearchCV(pipe_rdfor, dict( rd_for__estimator__ccp_alpha = alphas ), cv=5, scoring ='r2', n_jobs=-1)

start_time = time.time()
rdforCV.fit(X_train, y_train)
train_predict, test_predict, train_accuracy, test_accuracy, train_MSE, test_MSE = compute_metrics(rdforCV, X_train, y_train, X_test, y_test)
CV_times.append(time.time() - start_time)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))

r2.append(test_accuracy)
MSE.append(test_MSE)

#### Gradient Boosting Regressor
Και αυτή η τεχνική βασίζεται όπως είδαμε σε δέντρα, επομένως είναι λογικό να έχουμε πολύ υχηλή απόδοση εξαρχής. Εν τέλη βλέπουμε ότι μετά το τιουνάρισμα δεν έχουμε μεγάλη αύξηση της απόδοσης.

In [ ]:
GBR = MultiOutputRegressor(GradientBoostingRegressor(random_state=0))
start_time = time.time()
GBR.fit(X_train, y_train)
compute_metrics(GBR, X_train, y_train, X_test, y_test)
times.append(time.time() - start_time)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))

In [ ]:
alphas = np.arange(0.0,1.0,0.1)
pipe_gbr = Pipeline(steps=[('scaler', scaler), ('GBR', GBR)], memory = 'tmp')
gbrCV = GridSearchCV(pipe_gbr, dict( GBR__estimator__alpha = alphas ), cv=5, scoring ='r2', n_jobs=-1)

start_time = time.time()
gbrCV.fit(X_train, y_train)
train_predict, test_predict, train_accuracy, test_accuracy, train_MSE, test_MSE = compute_metrics(gbrCV, X_train, y_train, X_test, y_test)
CV_times.append(time.time() - start_time)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))

r2.append(test_accuracy)
MSE.append(test_MSE)

In [ ]:
GBR.get_params().keys()

## Time and Metric Plots
Στην συνέχεια κάνουμε τα bar plots που ζητούνται στην άσκηση. Πρώτα από όλα συγκρίνουμε τους χρόνους που τρέχει ο κάθε ένα από τους αλγορίθμους και στην συνέχεια ο χρόνος που χρειάζεται να κάνουμε το grid search. Βεβαίως αυτός ο χρόνος εξαρτάται από το πλήθος των παραμέτρων που θέλουμε να τιουνάρουμε. Εμείς αναφερόμαστε στο τιουνάρισμα που καναμε εμείς.

Βλέπουμε ότι γενικά τα boosting-ensemble μοντέλα χρειάζονται του μεγαλύτερους χρόνους.

In [ ]:
import seaborn as sns

plot_data = {'names': names, 'times': times}

sns.set_theme(style="whitegrid")
tips = sns.load_dataset("tips")
ax = sns.barplot(x="names", y="times", data=plot_data).set_title("Algorithm Times Barplot",fontsize=16)

In [ ]:
plot_data = {'names': names, 'tuning times': CV_times}

sns.set_theme(style="whitegrid")
tips = sns.load_dataset("tips")
ax = sns.barplot(x="names", y="tuning times", data=plot_data).set_title("Algorithm Tuning Times Barplot",fontsize=16)

Όσον αφορά την απόδοση των ταξινομητών βλέπουμε ότι τα δάση έχουν την καλύτερη δυνατή απόδοση. Παρόλα αυτά, όλοι οι αλγόριθμοι έχουν γενικά καλή απόδοση, δεν υπάρχει κάποιος που να μας δίνει κακά αποτελέσματα.

Ίσως να είναι προτιμότεροι οι τέσσερεις τελευταίοι που έχουν και το μικρότερο μέσο τετραγωνικό σφάλμα.

In [ ]:
plot_data = {'names': names, 'R2 metric': r2}

sns.set_theme(style="whitegrid")
tips = sns.load_dataset("tips")
ax = sns.barplot(x="names", y="R2 metric", data=plot_data).set(title='R2 metric bar plot', ylim=(0.9, 1.0))

In [ ]:
plot_data = {'names': names, 'MSE metric': MSE}

sns.set_theme(style="whitegrid")
tips = sns.load_dataset("tips")
ax = sns.barplot(x="names", y="MSE metric", data=plot_data).set_title("MSE metric Barplot",fontsize=16)